In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# to read cvs faster (uses GPU)
import cudf

# to split dataset
from sklearn.model_selection import train_test_split
# model
from xgboost import XGBClassifier
# to split dataset in folds for cross-validation preserving the percentage of samples for each class
from sklearn.model_selection import StratifiedKFold
# to perform a randomized search for cross-validation
from sklearn.model_selection import RandomizedSearchCV
# to calculate the score
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.feature_selection import mutual_info_classif

# to perform a hyperparameter scan using Bayesian Optimization
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

import matplotlib.pyplot as plt

# garbage collector: to free-up memory when needed
import gc

# to keep track of time
import time


from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA

from sklearn.cluster import KMeans

In [ ]:
%%time
# Loading data sets using cudf (faster) and coverts to pandas (DataFrame)
train = cudf.read_csv('../input/tabular-playground-series-nov-2021/train.csv', index_col = 'id').to_pandas()
train

In [ ]:
# All features are continuous, and some seem to need standardisation
# First let us separate X from y and divide the sets into search set - to perform a randomized search -
# and opt set - to be fitted by the model with the best "optimized hyperparameter set" from the search.
X = train.drop(columns='target')
y = train.target
X_search, X_opt, y_search, y_opt = train_test_split(X, y, train_size = 0.5, random_state=7)

# since the datasets below won't be used here, we free up memory space by removing them
del train
del X
del y
del X_opt
del y_opt
gc.collect()


In [ ]:
# Scaling and finding Principal Components
# sc = StandardScaler()
# prep = ColumnTransformer([('sc', sc, X_search.columns)], remainder='passthrough')

# X_search = pd.DataFrame(prep.fit_transform(X_search), index=X_search.index, columns=X_search.columns)

pca = PCA()
X_search = pd.DataFrame(pca.fit_transform(X_search))

# wcss = []
# for i in range(1, 151, 25):
#   kmeans = KMeans(n_clusters=i, init = 'k-means++', random_state=7)
#   kmeans.fit(X_search)
#   wcss.append(kmeans.inertia_)

# plt.plot(range(1, 151, 25), wcss)
# plt.title('The Elbow Method')
# plt.xlabel('Number of Clusters')
# plt.ylabel('WCSS')
# plt.show()

In [ ]:
kmeans = KMeans(n_clusters=60,  init = 'k-means++', random_state=7)
X_search["Cluster"] = kmeans.fit_predict(X_search)

# discrete = X_search.dtypes == 'int'

# mi_scores = pd.Series(mutual_info_classif(X_search, y_search, discrete_features=discrete, random_state=7), index=X_search.columns).sort_values(ascending=False)
# mi_scores

In [ ]:
# Defining the model: 'gpu_hist' is important to run it faster with GPU
model = XGBClassifier(tree_method='gpu_hist', use_label_encoder=False, eval_metric='auc', random_state=7)

In [ ]:
# Benchmark, adapted from TPC Oct 21 (randomized search, no scaling)
# Best: 0.740070 using {'subsample': 0.30000000000000004, 'n_estimators': 800, 'max_depth': 3, 'learning_rate': 0.07600000000000001}
# Seconds to run the scan: 1208.465334


# Let us vary thorough the XGBoost paramenters to see which setup gives the best result (score)

start = time.time()

# define the hyperparameters and the ranges to perform the scan
# params_rnd = {'n_estimators':np.arange(100, 1000, 100),'learning_rate':np.arange(0.01, 0.31, 0.01),
#           'max_depth':np.arange(3, 12, 1), 'subsample':np.arange(0.1, 1, 0.1), 'colsample_bytree':np.arange(0.1, 1.1, 0.1),
#          'colsample_bylevel':np.arange(0.1, 1.1, 0.1), 'min_child_weight':np.arange(0,10,1), 'reg_alpha':np.arange(0,15,1), 
#               'reg_lambda':np.arange(0,30,1)}


params_rnd = {'n_estimators':np.arange(500, 1200, 50),'learning_rate':np.arange(0.006, 0.21, 0.01),
          'max_depth':np.arange(3, 12, 1), 'subsample':np.arange(0.1, 1, 0.1)}

# for cross validation with 5 splits, using StratifiedKFold to keep the same percentage of sample per each class
skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)

# defining the random scan, n_iter=7 (picks 7 random scenarios from params_rnd), using 'roc_auc' and the scoring method
rnd_search = RandomizedSearchCV(model, params_rnd, n_iter=30, scoring='roc_auc', cv=skfold, random_state=7)

# the model should fit the search set
rnd_result = rnd_search.fit(X_search, y_search)

# print the best score in the search and the corresponding best parameters
print("Best: %f using %s" % (rnd_result.best_score_, rnd_result.best_params_))

# print the total time...
elapsed = time.time() - start
print("Seconds to run the scan: %f" % (elapsed))

# Search datasets won't be used anymore, so we remove them from memory
del X_search
del y_search
gc.collect()

In [ ]:
# now, using the tuned hyperparameters, we fit and test the model on the "opt" set 
# Defining the model
model_opt = XGBClassifier(**rnd_result.best_params_, tree_method='gpu_hist', use_label_encoder=False, 
                          eval_metric='auc', random_state=7)

# pipe_opt = Pipeline(steps=[('preprocessing', prep), ('model', model_opt)])

train = cudf.read_csv('../input/tabular-playground-series-nov-2021/train.csv', index_col = 'id').to_pandas()
X = train.drop(columns='target')
y = train.target
X_search, X_opt, y_search, y_opt = train_test_split(X, y, train_size = 0.5, random_state=7)

# X_opt = pd.DataFrame(prep.transform(X_opt), index=X_opt.index, columns=X_opt.columns)

X_opt = pd.DataFrame(pca.transform(X_opt))

X_opt["Cluster"] = kmeans.predict(X_opt)

# Search datasets won't be used anymore, so we remove them from memory
del X
del y
del X_search
del y_search
gc.collect()

model_opt.fit(X_opt, y_opt)

del X_opt
del y_opt
gc.collect()

In [ ]:
### We calculate and store the probability of the positive prediction

X_test = cudf.read_csv('../input/tabular-playground-series-nov-2021/test.csv', index_col = 'id').to_pandas()


# X_test = pd.DataFrame(prep.transform(X_test), index=X_test.index, columns=X_test.columns)

X_test_2 = pd.DataFrame(pca.transform(X_test))


X_test_2["Cluster"] = kmeans.predict(X_test_2)

pred_test = model_opt.predict_proba(X_test_2)[:,1]
# pred_test = pipe_opt.predict_proba(X_test_kpca)[:,1]


output = pd.DataFrame({'id': X_test.index,
                       'target': pred_test})
output.to_csv('submission_TPSNov21.csv', index=False)

In [ ]:
X_test

In [ ]:
output